Adaptation of the script made to fetch swiss companies to fetch other countries as well.

For this purpose, we fetch the page using requests, and then parse its content using BeautifulSoup.

In [5]:
import requests
from bs4 import BeautifulSoup
import csv

In [72]:
def get_companies(coutry):
    r = requests.get('https://en.wikipedia.org/wiki/List_of_companies_of_' + country)
    if r.status_code != 200:
        print('HTTP ERROR ' + r.status_code + ' country: ' + country)
        return []
    
    soup = BeautifulSoup(r.text, 'html.parser')

    content = soup.find_all(id = 'mw-content-text')
    if len(content) < 1:
        print("No companies found in " + country)
        return []

    companies = []
    for x in content[0].find_all('h2'):
        nxt = x.find_next_sibling('ul')
        if nxt == None:
            continue
            
        for y in nxt.find_all('a'):
            companies.append(y.string)

    print(str(len(companies)) + ' companies found in ' + country)
    return companies

def to_csv(country, companies):
    with open('companies/' + country + '_companies.csv', 'w') as f:
        wr = csv.writer(f, quoting=csv.QUOTE_ALL)
        wr.writerow(companies)

In [40]:
def get_french_companies():
    r = requests.get('https://en.wikipedia.org/wiki/List_of_companies_of_France')
    if r.status_code != 200:
        print('HTTP ERROR ' + r.status_code + ' country: ' + country)
        return []
    
    soup = BeautifulSoup(r.text, 'html.parser')

    content = soup.find_all(id = 'mw-content-text')
    if len(content) < 1:
        print("No companies found in " + country)
        return []

    companies = []
    for x in content[0].find_all('h2'):
        nxt = x.find_next_sibling('div')
        if nxt == None:
            continue
            
        nxt = nxt.find('table', class_='multicol')
        if nxt == None:
            continue
            
        for y in nxt.find_all('a'):
            companies.append(y.string)

    print(str(len(companies)) + ' french companies found')
    return companies

In [68]:
def get_uk_companies():
    r = requests.get('https://en.wikipedia.org/wiki/List_of_largest_private_companies_in_the_United_Kingdom')
    if r.status_code != 200:
        print('HTTP ERROR ' + r.status_code + ' country: ' + country)
        return []
    
    soup = BeautifulSoup(r.text, 'html.parser')

    content = soup.find(id = 'mw-content-text').find('table', class_ = 'wikitable')

    companies = []
    for tr in content.find_all(lambda el: el.name == 'tr' and el.td != None):
        try:
            companies.append(tr.td.a.string)
        except AttributeError:
            companies.append(tr.td.string)

    print(str(len(companies)) + ' companies found in the uk')
    return companies

In [70]:
countries = ['Germany',
            'Italy',
            'Spain'] #UK and France have special treatment because their wikipedia pages are not formatted the same

In [73]:
for country in countries:
    companies = get_companies(country)
    to_csv(country, companies)

347 companies found in Germany
586 companies found in Italy
151 companies found in Spain


In [74]:
to_csv("France", get_french_companies())

212 french companies found


In [75]:
to_csv("UK", get_uk_companies())

99 companies found in the uk
